<a href="https://colab.research.google.com/github/nicholasfalconi/CMPE-320/blob/master/melanoma_benign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Benign or Malignant skin cancer detection model
___________________________________________________________

# Nicholas Falconi, 2022

##Current Architecture is U-Net ++ 

# Mount Gooogle Drive ( Data )

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [ ]:
import os
import shutil
from tqdm import tqdm
import pandas as pd

# Imports for Tensorflow 
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Set the paths for the train and test csv files
train_csv_path = "/content/drive/MyDrive/Mole_AI/train.csv"
test_csv_path = "/content/drive/MyDrive/Mole_AI/test.csv"

# Data frames for train and test labels
trainLabels = pd.read_csv(train_csv_path)
trestLabels = pd.read_csv(test_csv_path)

benign_malignant = trainLabels[['image_name', 'benign_malignant']]

# Load csv into data-frames

# Seperate training data into malignant & Benign folders

In [ ]:
'''
Seperate training images into two folders in the following format

Mole_AI ----- test ---- train --- train.csv --- test.csv
                          |
                          |
             malignant____|____ benign
                |                 |
                img1.jpg          img1.jpg
                img2.jpg          img2.jpg
                ...               ...
'''

# Get Benign file names
benign_malignant = trainLabels[['image_name', 'benign_malignant']]

# Creating Dataframe
bm_df = pd.DataFrame(benign_malignant, columns = ['image_name', 'benign_malignant'])

# Benign 
benign = bm_df[bm_df['benign_malignant'] == 'benign']

# Malignant 
malignant = bm_df[bm_df['benign_malignant'] == 'malignant']

# Move files for Benign
for name in tqdm(benign.image_name):
  shutil.move("/content/drive/MyDrive/Mole_AI/train/train/{}.jpg".format(name), "/content/drive/MyDrive/Mole_AI/training/benign")


# Move files for malignant
for name in tqdm(malignant.image_name):
  shutil.move("/content/drive/MyDrive/Mole_AI/train/train/{}.jpg".format(name), "/content/drive/MyDrive/Mole_AI/training/malignant")


#Define Model

In [39]:
"""
Update this model to U-net
"""

model = tf.keras.models.Sequential([
                                    
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),

    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('benign') and 1 for the other ('malignant')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

# Compile model

In [41]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

# Pre-process images 

In [42]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Mole_AI/training',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary',
        horizontal_flip=True,
        vertical_flip=True,
        featurewise_std_normalization=True)


Found 33126 images belonging to 2 classes.


# Train the model

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)

Epoch 1/15
8/8 [==============================] - 540s 66s/step - loss: 0.2930 - accuracy: 0.9756
Epoch 2/15
8/8 [==============================] - 543s 67s/step - loss: 0.0921 - accuracy: 0.9824
Epoch 3/15
8/8 [==============================] - 567s 70s/step - loss: 0.1073 - accuracy: 0.9805
Epoch 4/15
3/8 [==========>...................] - ETA: 5:49 - loss: 0.0909 - accuracy: 0.9792